In [ ]:
import os
import requests
import pandas as pd
import time

def fetch_questions_and_answers(keywords, save_directory):
    try:
        all_data = []

        for keyword in keywords:
            params = {
                'site': 'stackoverflow',
                'q': keyword,
                'sort': 'votes',  # Sort by votes to get popular questions first
                'order': 'desc',
                'pagesize': 100  # Max number of questions per page
            }

            page = 1

            while True:
                params['page'] = page
                response = requests.get('https://api.stackexchange.com/2.3/search/advanced', params=params)
                print(f"Response status for page {page} with keyword '{keyword}': {response.status_code}")  # Print the response status

                response.raise_for_status()  # This will raise an exception for HTTP errors
                data = response.json()

                if 'items' in data:
                    questions = data['items']

                    for question in questions:
                        question_id = question['question_id']
                        question_title = question['title']
                        question_body = question.get('body', '')
                        question_score = question['score']
                        question_view_count = question['view_count']
                        question_creation_date = question['creation_date']
                        question_last_activity_date = question['last_activity_date']

                        answers = fetch_answers(question_id)
                        answer_data = [{
                            "Answer Body": answer.get('body', ''),
                            "Answer Score": answer['score'],
                            "Answer Creation Date": answer['creation_date'],
                            "Answer Last Activity Date": answer['last_activity_date']
                        } for answer in answers]

                        all_data.append({
                            "Keyword": keyword,
                            "Question Title": question_title,
                            "Question Body": question_body,
                            "Question Score": question_score,
                            "Question View Count": question_view_count,
                            "Question Creation Date": question_creation_date,
                            "Question Last Activity Date": question_last_activity_date,
                            "Answers": answer_data
                        })

                    if len(questions) < 100:
                        # Less than 100 items on this page means it's the last page
                        break
                    else:
                        # Increment page number for next request
                        page += 1
                else:
                    # No more items found
                    break

        if all_data:
            df = pd.DataFrame(all_data)
            file_name = f"StackOverflowKeywords.csv"
            # Save DataFrame to CSV in my working directory
            df.to_csv(os.path.join(save_directory, file_name), index=False)
            print(f"Exported to {os.path.join(save_directory, file_name)}")
        else:
            print("No questions found.")

    except requests.RequestException as e:
        print(f"Request failed: {e}")
    except requests.HTTPError as e:
        print(f"HTTP error occurred: {e}")  # HTTP error occurred
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


def fetch_answers(question_id):
    params = {
        'site': 'stackoverflow',
        'order': 'desc',
        'sort': 'votes'
    }

    try:
        response = requests.get(f'https://api.stackexchange.com/2.3/questions/{question_id}/answers', params=params)
        response.raise_for_status()  # This will raise an exception for HTTP errors
        data = response.json()
        return data['items']

    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return []
    except requests.HTTPError as e:
        print(f"HTTP error occurred: {e}")  # HTTP error occurred
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []


# Example usage
keywords = ['After approval, Keywords will be listed here']  # Specify your keywords here
# Specify the directory where the CSV file will be saved
current_directory = os.getcwd()
fetch_questions_and_answers(keywords, current_directory)
